In [1]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

from flask import Flask, jsonify, request
from flask_cors import CORS, cross_origin

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

import pickle
import pandas as pd
import numpy as np
import tensorflow as tf
import random

Using TensorFlow backend.


In [2]:
# import our chat-bot intents file
import json
with open('intents.json') as json_data:
    data = json.load(json_data)

In [3]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in data['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique stemmed words", words)

45 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
82 unique stemmed words ["'s", ',', 'a', 'advers', 'al', 'anyon', 'ar', 'awesom', 'be', 'behavy', 'blood', 'by', 'bye', 'can', 'caus', 'chat', 'check', 'could', 'dat', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'giv', 'good', 'goodby', 'hav', 'hello', 'help', 'hi', 'hist', 'hospit', 'how', 'i', 'id', 'is', 'lat', 'list', 'load', 'loc', 'log', 'look', 'lookup', 'man', 'me', 'mod', 'nearby', 'next', 'nic', 'of', 'off', 'op', 'paty', 'pharm', 'press', 'provid', 'react', 'rel', 'result', 'search', 'see', 'show', 'suit', 'support', 'task', 'thank', 'that', 'ther', 'til', 'tim', 'to', 'transf', 'up', 'want', 'what', 'which', 'with', 'you']


In [4]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word - create base word, in attempt to represent related words
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])

In [5]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [6]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [7]:
# Fit the model
model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
45/45 [==============================] - 0s 3ms/step - loss: 2.2588 - accuracy: 0.1111
Epoch 2/200
45/45 [==============================] - 0s 325us/step - loss: 2.2074 - accuracy: 0.1333
Epoch 3/200
45/45 [==============================] - 0s 315us/step - loss: 1.9969 - accuracy: 0.4222
Epoch 4/200
45/45 [==============================] - 0s 308us/step - loss: 2.0023 - accuracy: 0.3111
Epoch 5/200
45/45 [==============================] - 0s 226us/step - loss: 1.8624 - accuracy: 0.4444
Epoch 6/200
45/45 [==============================] - 0s 246us/step - loss: 1.8644 - accuracy: 0.4667
Epoch 7/200
45/45 [==============================] - 0s 253us/step - loss: 1.6918 - accuracy: 0.5111
Epoch 8/200
45/45 [==============================] - 0s 232us/step - loss: 1.6725 - accuracy: 0.4444
Epoch 9/200
45/45 [==============================] - 0s 215us/step - loss: 1.3642 - accuracy: 0.5778
Epoch 10/200
45/45 [==============================] - 0s 246us/step - loss: 1.2829 - accuracy

45/45 [==============================] - 0s 234us/step - loss: 0.0464 - accuracy: 0.9778
Epoch 82/200
45/45 [==============================] - 0s 222us/step - loss: 0.0569 - accuracy: 1.0000
Epoch 83/200
45/45 [==============================] - 0s 221us/step - loss: 0.0140 - accuracy: 1.0000
Epoch 84/200
45/45 [==============================] - 0s 246us/step - loss: 0.0463 - accuracy: 0.9778
Epoch 85/200
45/45 [==============================] - 0s 231us/step - loss: 0.0730 - accuracy: 1.0000
Epoch 86/200
45/45 [==============================] - 0s 232us/step - loss: 0.0605 - accuracy: 0.9556
Epoch 87/200
45/45 [==============================] - 0s 233us/step - loss: 0.0539 - accuracy: 0.9778
Epoch 88/200
45/45 [==============================] - 0s 220us/step - loss: 0.0108 - accuracy: 1.0000
Epoch 89/200
45/45 [==============================] - 0s 213us/step - loss: 0.0532 - accuracy: 1.0000
Epoch 90/200
45/45 [==============================] - 0s 260us/step - loss: 0.0327 - accuracy: 

45/45 [==============================] - 0s 211us/step - loss: 0.0483 - accuracy: 1.0000
Epoch 161/200
45/45 [==============================] - 0s 228us/step - loss: 0.0239 - accuracy: 1.0000
Epoch 162/200
45/45 [==============================] - 0s 202us/step - loss: 0.0234 - accuracy: 1.0000
Epoch 163/200
45/45 [==============================] - 0s 214us/step - loss: 0.0274 - accuracy: 1.0000
Epoch 164/200
45/45 [==============================] - 0s 217us/step - loss: 0.0044 - accuracy: 1.0000
Epoch 165/200
45/45 [==============================] - 0s 245us/step - loss: 0.0115 - accuracy: 1.0000
Epoch 166/200
45/45 [==============================] - 0s 232us/step - loss: 0.0217 - accuracy: 1.0000
Epoch 167/200
45/45 [==============================] - 0s 248us/step - loss: 0.0130 - accuracy: 1.0000
Epoch 168/200
45/45 [==============================] - 0s 245us/step - loss: 0.0118 - accuracy: 1.0000
Epoch 169/200
45/45 [==============================] - 0s 254us/step - loss: 0.0235 - a

In [8]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [9]:
p = bow("Load blood pessure for patient", words)
print (p)
print (classes)

found in bag: load
found in bag: blood
found in bag: for
found in bag: paty
[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0]
['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']


In [10]:
inputvar = pd.DataFrame([p], dtype=float, index=['input'])

print(model.predict(inputvar))

[[2.3305653e-04 3.7428883e-05 9.9948102e-01 5.9611139e-05 2.4377887e-05
  1.4057003e-04 2.4866129e-06 1.2838283e-05 8.5468237e-06]]


In [11]:
from tensorflow.python.framework import ops

global graph
graph = ops.get_default_graph()


In [12]:
def classify_local(sentence):
    ERROR_THRESHOLD = 0.25
    
    # generate probabilities from the model
    input_data = pd.DataFrame([bow(sentence, words)], dtype=float, index=['input'])
    results = model.predict([input_data])[0]
    # filter out predictions below a threshold, and provide intent index
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], str(r[1])))
    # return tuple of intent and probability
    
    return return_list

In [13]:
classify_local('Hello, good day!')

found in bag: hello
found in bag: ,
found in bag: good
found in bag: day


[('greeting', '0.9999933')]

In [14]:
classify_local('How you can assist me?')

found in bag: how
found in bag: you
found in bag: can
found in bag: me


[('options', '0.9997876')]

In [16]:
#Testing speach recognition
import speech_recognition as sr
r = sr.Recognizer()
mic = sr.Microphone()
print('Say somehting now')
with mic as source:
    r.adjust_for_ambient_noise(source)
    audio = r.listen(source)
    transcript = r.recognize_google(audio)
    print(transcript)

Say somehting now
hello


In [17]:
#Testing audio output
import subprocess
def say(text):
    subprocess.call(['say', text])
say("Hello Ralph")

In [18]:
classify_local(transcript)

found in bag: hello


[('greeting', '0.99902')]

In [19]:
r = sr.Recognizer()
mic = sr.Microphone()

def chat():
    print("Say somehting now!")
    with mic as source:
        r.adjust_for_ambient_noise(source)
        audio = r.listen(source)
        transcript = r.recognize_google(audio)
        print(transcript)
    
    # generate probabilities from the model
    input_data = pd.DataFrame([bow(transcript, words)], dtype=float, index=['input'])
    results = model.predict([(input_data)])[0]
    results_index = np.argmax(results)
    tag = classes[results_index]
        
    if(results[results_index] > 0.5):
        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']
        say(random.choice(responses))
    else:
        say(" not sure what you mean. Try again")

In [22]:
chat()

Say somehting now!
how can you assist me
found in bag: how
found in bag: can
found in bag: you
found in bag: me
